# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries and watermark our file - to show what libraries and versions we're using. Then define utility functions to integrate with our Object storage and _Verta_ visualisation server.

In [6]:
import os
# os.environ["MODIN_ENGINE"] = "ray"
%pip install matplotlib
%pip install pandas
%pip install sklearn
%pip install minio
%pip install seaborn
%pip install mlflow
%pip install ipynbname
%pip install category_encoders
%pip install joblib
%pip install watermark


You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the 

In [7]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
# import pandas as pd
# import modin.pandas as pd

import watermark
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
from minio import Minio
from minio.error import ResponseError
import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline


# import tools as tools
%matplotlib inline
%load_ext watermark

In [8]:
%watermark -n -v -m -g -iv


Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.28.0

Compiler    : GCC 8.4.1 20200928 (Red Hat 8.4.1-1)
OS          : Linux
Release     : 4.18.0-305.25.1.el8_4.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit

Git hash: 828398003b6da4f9e981d1fb40477e389dfa52d4

sklearn   : 0.0
matplotlib: 3.5.0
pandas    : 1.3.4
seaborn   : 0.11.2
watermark : 2.2.0
numpy     : 1.21.4



### In this next section, on the third line, change experiment_name by appending your username to _customerchurn_, e.g., if your username is user1: 
#### experiment_name = "customerchurnuser1"

In [12]:
HOST = "http://mlflow:5500"

PROJECT_NAME = "CustomerChurnUser29"
EXPERIMENT_NAME = "DecisionTreeClassifierv6"

os.environ['MLFLOW_S3_ENDPOINT_URL']='minio-ml-workshop:9000'
os.environ['AWS_ACCESS_KEY_ID']='minio'
os.environ['AWS_SECRET_ACCESS_KEY']='minio123'
os.environ['AWS_REGION']='us-east-1'
os.environ['AWS_BUCKET_NAME']='mlflow'

dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_name = "customerchurnuser29"
experiment_id = experiment_name + timestampStr

def get_s3_server():
    minioClient = Minio(os.environ['MLFLOW_S3_ENDPOINT_URL'],
                    access_key=os.environ['AWS_ACCESS_KEY_ID'],
                    secret_key=os.environ['AWS_SECRET_ACCESS_KEY'],
                    secure=False)

    return minioClient






### In this next section, on the second line, insert the value you retrieved from Minio object storage earlier - representing the fully qualified name of your csv file in Minio. This is the file pushed by the data engineer in the format: full_data_csv{USERNAME}/{FILENAME}.csv. 
#### In my case this value is: full_data_csvuser29/part-00000-59149e08-583c-46a5-bfa0-0b3abecbf1a3-c000.csv (yours will be different)
### We refer to this fully qualified name in the Github instructions as CSV-FILE

In [14]:
minioClient = get_s3_server()
data_file = minioClient.fget_object("data", "full_data_csvuser29/part-00000-ef66cc0f-e3e8-476d-a2b3-7101e59f2f09-c000.csv", "/tmp/data.csv")
data_file_version = data_file.version_id
data = pd.read_csv('/tmp/data.csv')
data.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,Premium,RelationshipManager,PrimaryChannel,HasCreditCard,...,IncomeProtection,WealthManagement,HomeEquityLoans,MoneyMarketAccount,CreditRating,PaperlessBilling,AccountType,MonthlyCharges,TotalCharges,Churn
0,148,Male,0,No,No,1,Yes,No,Mobile,No,...,No,No,No,No,High,Yes,Checking,45.65,45.65,Yes
1,463,Male,0,Yes,Yes,4,Yes,Yes,Branch,No,...,Yes,No,Yes,Yes,High,No,Checking,101.15,385.90,Yes
2,471,Female,1,No,No,17,Yes,No,No,Not Available,...,Not Available,Not Available,Not Available,Not Available,Medium,No,Savings,20.65,330.60,No
3,496,Male,0,No,No,22,No,Not available,Mobile,No,...,Yes,No,No,Yes,Medium,Yes,Brokerage,43.75,903.60,Yes
4,833,Female,0,Yes,Yes,70,Yes,No,Mobile,Yes,...,Yes,Yes,No,Yes,Medium,No,Credit Card,74.10,5222.30,No


### Use pandas.DataFrame functions
- _shape_ to return the dimensionality
- _info_ to print a concise summary of the DataFrame
- _describe_ to generate descriptive statistics of the DataFrame's columns
- _isnull().sum()_ to sum the empty values
- finally determine Churn and Total Changes 


In [15]:
data.shape

(7043, 21)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customerID           7043 non-null   int64  
 1   gender               7043 non-null   object 
 2   SeniorCitizen        7043 non-null   int64  
 3   Partner              7043 non-null   object 
 4   Dependents           7043 non-null   object 
 5   tenure               7043 non-null   int64  
 6   Premium              7043 non-null   object 
 7   RelationshipManager  7043 non-null   object 
 8   PrimaryChannel       7043 non-null   object 
 9   HasCreditCard        7043 non-null   object 
 10  DebitCard            7043 non-null   object 
 11  IncomeProtection     7043 non-null   object 
 12  WealthManagement     7043 non-null   object 
 13  HomeEquityLoans      7043 non-null   object 
 14  MoneyMarketAccount   7043 non-null   object 
 15  CreditRating         7043 non-null   o

In [17]:
data.describe()

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000,7032.000000
mean,3522.000000,0.162147,32.371149,64.761692,2283.300441
std,2033.283305,0.368612,24.559481,30.090047,2266.771362
min,1.000000,0.000000,0.000000,18.250000,18.800000
25%,1761.500000,0.000000,9.000000,35.500000,401.450000
50%,3522.000000,0.000000,29.000000,70.350000,1397.475000
75%,5282.500000,0.000000,55.000000,89.850000,3794.737500
max,7043.000000,1.000000,72.000000,118.750000,8684.800000


In [18]:
data.isnull().sum()

customerID              0
gender                  0
SeniorCitizen           0
Partner                 0
Dependents              0
tenure                  0
Premium                 0
RelationshipManager     0
PrimaryChannel          0
HasCreditCard           0
DebitCard               0
IncomeProtection        0
WealthManagement        0
HomeEquityLoans         0
MoneyMarketAccount      0
CreditRating            0
PaperlessBilling        0
AccountType             0
MonthlyCharges          0
TotalCharges           11
Churn                   0
dtype: int64

In [19]:
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

In [20]:
data.replace(" ", np.nan, inplace=True)

In [21]:
data.isna().sum()

customerID              0
gender                  0
SeniorCitizen           0
Partner                 0
Dependents              0
tenure                  0
Premium                 0
RelationshipManager     0
PrimaryChannel          0
HasCreditCard           0
DebitCard               0
IncomeProtection        0
WealthManagement        0
HomeEquityLoans         0
MoneyMarketAccount      0
CreditRating            0
PaperlessBilling        0
AccountType             0
MonthlyCharges          0
TotalCharges           11
Churn                   0
dtype: int64

In [22]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

In [23]:
mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
# Now we know that total charges has nan values
data.isna().sum()

customerID             0
gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
tenure                 0
Premium                0
RelationshipManager    0
PrimaryChannel         0
HasCreditCard          0
DebitCard              0
IncomeProtection       0
WealthManagement       0
HomeEquityLoans        0
MoneyMarketAccount     0
CreditRating           0
PaperlessBilling       0
AccountType            0
MonthlyCharges         0
TotalCharges           0
Churn                  0
dtype: int64

## Feature Engineering pipeline
### Use category_encoder's Ordinal encoding method which uses a single column of integers to represent the classes - then fit that to our 2 dimensional data imported earlier. Then pickle it and transform it. Then use Onehot (or dummy) coding for categorical features, producing one feature per category, each binary.


In [24]:
import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
# for column in names:
#     labelencoder(column)

enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data)
joblib.dump(enc, 'enc.pkl')
labelled_set = enc.transform(data)
labelled_set.tail(5)

KeyError: 'PhoneService'

In [ ]:

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
ohe.fit(labelled_set)
joblib.dump(ohe, 'ohe.pkl')
final_set = ohe.transform(labelled_set)
final_set.tail(5)

### Now we use scikit-learn's 'train_test_split' function to randomly split our data into training and testing sets. Then remove the _Churn_ and _customerID_ fields from our training and testing datasets and output the shaope of our data.

In [ ]:
labels = final_set['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
X_train.pop('Churn')
X_train.pop('customerID')
X_test.pop('Churn')
X_test.pop('customerID')
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

In [ ]:

# Data For cross validation and GridSearch
Y = final_set['Churn']
X = final_set.drop(['Churn', 'customerID'], axis=1)
print ('Training Data Shape', X.shape)
print ('Testing Data Shape', Y.shape)

### Create DecisionTreeClassifier object, extract hyper parameters, and then GridSearch will best_model from the various inputs

In [ ]:
# Create decision tree object
DT = DecisionTreeClassifier()
# List of parameters
# entropy
criterion = ['gini']
max_depth = [5,10,15]
min_samples_split = [2,4,6]
min_samples_leaf = [4,5,6,8]
# Save all the lists in the variable
hyperparameters = dict(max_depth=max_depth, criterion=criterion,min_samples_leaf = min_samples_leaf ,min_samples_split = min_samples_split)

In [ ]:
model = GridSearchCV(DT, hyperparameters, cv=5, verbose=0)
best_model = model.fit(X,Y)

In [ ]:
# Mean cross validated score
print('Mean Cross-Validated Score: ',best_model.best_score_)
print('Best Parameters',best_model.best_params_)
# You can also print the best penalty and C value individually from best_model.best_estimator_.get_params()
print('Best criteria:', best_model.best_estimator_.get_params()['criterion'])
print('Best depth:', best_model.best_estimator_.get_params()['max_depth'])

### Use K-Folds cross-validator to split data in train/test sets. Create a dictionary of hyperparameter candidates, train model using a DecisionTreeClassifier, assess results, print and store hyper parameters and accuracy and tag using 'DecisionTreeClassifier'


In [ ]:
kfold = KFold(n_splits = 3)
hyperparameters = dict(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
model = DecisionTreeClassifier(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
model = model.fit(X_train, y_train)
joblib.dump(model, 'dct.pkl')
results = model_selection.cross_val_score(model,X,Y,cv = kfold)
print(results)
print('Accuracy',results.mean()*100)
store = get_meta_store()
store.log_hyperparameters(hyperparameters)
store.log_model(model)
store.log_metric('Accuracy',results.mean()*100)
store.log_tag("DecisionTreeClassifier")
# get_meta_store().log_dataset_version("raw_data", dataset_version)

### Like before, in this next section, on the third line, change experiment_name by appending your username to _customerchurn_, e.g., if your username is user1: 
#### experiment_name = "customerchurnuser1"
### Create RandomForestClassifier object, extract hyper parameters, and then the best_model

In [ ]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d%Y%H%M%S%f")
experiment_name = "customerchurnuser29"
experiment_id = experiment_name + timestampStr


# Create random forest object
RF = RandomForestClassifier()
n_estimators = [18,22]
criterion = ['gini', 'entropy']
# Create a list of all of the parameters
max_depth = [30,40,50]
min_samples_split = [6,8]
min_samples_leaf = [8,10,12]
# Merge the list into the variable
hyperparameters = dict(n_estimators = n_estimators,max_depth=max_depth, criterion=criterion,min_samples_leaf = min_samples_leaf ,min_samples_split = min_samples_split)
# Fit your model using gridsearch
model = GridSearchCV(RF, hyperparameters, cv=5, verbose=0)
best_model = model.fit(X, Y)

### Extract best scores, params, criteria and depth from our model. 

In [ ]:
# Mean cross validated score
print('Mean Cross-Validated Score: ',best_model.best_score_)
print('Best Parameters',best_model.best_params_)
# You can also print the best penalty and C value individually from best_model.best_estimator_.get_params()
print('Best criteria:', best_model.best_estimator_.get_params()['criterion'])
print('Best depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best estimator:', best_model.best_estimator_.get_params()['n_estimators'])


### As above, use K-Folds cross-validator to split data in train/test sets. Create a dictionary of hyperparameter candidates, train model using a RandomForestClassifier, assess results, print and store hyper parameters and accuracy and tag using 'RandomForestClassifier'

In [ ]:
kfold = KFold(n_splits = 3)
hyperparameters = dict(max_depth=40, criterion='gini',min_samples_leaf = 12 ,min_samples_split = 8, n_estimators = 22)
model = RandomForestClassifier(max_depth=40, criterion='gini',min_samples_leaf = 12 ,min_samples_split = 8, n_estimators = 22)
model = model.fit(X_train, y_train)
joblib.dump(model, 'rft.pkl')
results = model_selection.cross_val_score(model,X,Y,cv = kfold)
print(results)
print('Accuracy',results.mean()*100)
store = get_meta_store()
store.log_hyperparameters(hyperparameters)
store.log_model(model)
store.log_metric('Accuracy',results.mean()*100)
store.log_tag("RandomForestClassifier")
store.log_attribute("data_file_location", "data/full_data_csv/a.csv")
store.log_attribute("data_file_version", data_file_version)

In [ ]:
print('Notebook complete')